In [1]:
import polars as pl
import os
from pathlib import Path
import numpy as np

In [2]:
import Utils.constants
from Utils.constants import FOLDER
from Utils.constants import N_LANES, N_SPEEDS, N_TIME_LIMIT, N_DIRECTIONS, N_SPEED_DEVIATION, N_LANE_DEVIATION, N_TIME_LIMIT
from Utils.constants import MAX_COST_VALUE
from Utils.constants import index_to_speed,direction_to_lane_change
print(f"N_LANES: {N_LANES} N_SPEEDS:{N_SPEEDS} N_DIRECTIONS:{N_DIRECTIONS}")
print(f"State space {N_LANES*N_SPEEDS*N_DIRECTIONS} N_SPEED_DEVIATION:{N_SPEED_DEVIATION} N_LANE_DEVIATION:{N_LANE_DEVIATION}")

N_LANES: 15 N_SPEEDS:20 N_DIRECTIONS:3
State space 900 N_SPEED_DEVIATION:2 N_LANE_DEVIATION:1


In [3]:
#Frame wit the track information. To do: to move the creation of the lanes in here
path = Path(FOLDER + f"\\TrackLanes_{N_LANES}.parquet")
track_df = pl.read_parquet(path).limit(N_TIME_LIMIT)
N_TIME = len(track_df)
track_df

x_m,y_m,w_tr_right_m,w_tr_left_m,x_dir,y_dir,width,norm_dir,x_udir,y_udir,x_lane0,x_lane1,x_lane2,x_lane3,x_lane4,x_lane5,x_lane6,x_lane7,x_lane8,x_lane9,x_lane10,x_lane11,x_lane12,x_lane13,x_lane14,y_lane0,y_lane1,y_lane2,y_lane3,y_lane4,y_lane5,y_lane6,y_lane7,y_lane8,y_lane9,y_lane10,y_lane11,y_lane12,y_lane13,y_lane14,dir_delta
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3.439354,-0.495322,6.556,6.536,5.862596,8.10221,13.092,10.000792,0.586213,0.810157,8.742641,7.985028,7.227416,6.469804,5.712191,4.954579,4.196966,3.439354,2.681742,1.924129,1.166517,0.408904,-0.348708,-1.10632,-1.863933,-4.332673,-3.78448,-3.236287,-2.688094,-2.139901,-1.591708,-1.043515,-0.495322,0.052871,0.601064,1.149257,1.69745,2.245643,2.793836,3.342029,0.0
6.370784,3.555763,6.558,6.537,5.862596,8.10221,13.095,10.000792,0.586213,0.810157,11.675286,10.9175,10.159714,9.401928,8.644142,7.886356,7.12857,6.370784,5.612998,4.855212,4.097426,3.33964,2.581854,1.824068,1.066282,-0.282468,0.265851,0.81417,1.362488,1.910807,2.459126,3.007444,3.555763,4.104082,4.6524,5.200719,5.749038,6.297356,6.845675,7.393994,0.000046
9.30195,7.606888,6.56,6.538,5.862087,8.102292,13.098,10.00056,0.586176,0.810184,14.607844,13.849859,13.091874,12.333889,11.575905,10.81792,10.059935,9.30195,8.543965,7.78598,7.027995,6.270011,5.512026,4.754041,3.996056,3.768022,4.316432,4.864841,5.41325,5.96166,6.510069,7.058479,7.606888,8.155297,8.703707,9.252116,9.800526,10.348935,10.897344,11.445754,0.000043
12.232871,11.658055,6.562,6.539,5.861618,8.10238,13.101,10.000356,0.586141,0.810209,17.540146,16.781964,16.023782,15.2656,14.507417,13.749235,12.991053,12.232871,11.474689,10.716507,9.958325,9.200142,8.44196,7.683778,6.925596,7.818539,8.367041,8.915544,9.464046,10.012548,10.56105,11.109553,11.658055,12.206557,12.75506,13.303562,13.852064,14.400566,14.949069,15.497571,0.00004
15.163568,15.709268,6.564,6.54,5.861189,8.102473,13.104,10.00018,0.586108,0.810233,20.472213,19.713835,18.955457,18.197079,17.438701,16.680324,15.921946,15.163568,14.40519,13.646812,12.888435,12.130057,11.371679,10.613301,9.854923,11.869086,12.417684,12.966281,13.514878,14.063476,14.612073,15.160671,15.709268,16.257865,16.806463,17.35506,17.903658,18.452255,19.000852,19.54945,0.000037
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-11.222438,-20.750233,6.535,6.559,5.866447,8.101822,13.094,10.002736,0.586484,0.809961,-5.919626,-6.67717,-7.434715,-8.19226,-8.949804,-9.707349,-10.464893,-11.222438,-11.979983,-12.737527,-13.495072,-14.252616,-15.010161,-15.767706,-16.52525,-24.589945,-24.041415,-23.492885,-22.944354,-22.395824,-21.847294,-21.298763,-20.750233,-20.201703,-19.653172,-19.104642,-18.556112,-18.007581,-17.459051,-16.910521,0.000061
-8.289395,-16.699311,6.54,6.553,5.865724,8.101872,13.093,10.002352,0.586434,0.809997,-2.986752,-3.744272,-4.501793,-5.259313,-6.016834,-6.774354,-7.531875,-8.289395,-9.046915,-9.804436,-10.561956,-11.319477,-12.076997,-12.834518,-13.592038,-20.538404,-19.989962,-19.44152,-18.893079,-18.344637,-17.796195,-17.247753,-16.699311,-16.150869,-15.602427,-15.053985,-14.505543,-13.957102,-13.40866,-12.860218,0.00006
-5.356714,-12.648361,6.544,6.548,5.865019,8.10193,13.092,10.001986,0.586385,0.810032,-0.054244,-0.811739,-1.569235,-2.326731,-3.084227,-3.841722,-4.599218,-5.356714,-6.11421,-6.871706,-7.629201,-8.386697,-9.144193,-9.901689,-10.659184,-16.48684,-15.938486,-15.390132,-14.841778,-14.293424,-13.745069,-13.196715,-12.648361,-12.100007,-11.551653,-11.003298,-10.454944,-9.90659,-9.358236,-8.809882,0.000057


In [4]:
verbose = False
#lane transitions
def l0_range():
    return range(N_LANES)
def l1_range(index):
    return range(max(0,index-N_LANE_DEVIATION), min(N_LANES,index+N_LANE_DEVIATION+1))
distance_pairs = [(l0,l1) for l0 in l0_range() for l1 in l1_range(l0)]
if verbose:
    print("Distance pairs")
    for (l0,l1) in distance_pairs:
        print(f"{l0} -> {l1}")

def d_range(l0):
     return [index for index in range(N_DIRECTIONS) if (direction_to_lane_change(index) + l0 >= 0 and direction_to_lane_change(index) + l0 < N_LANES)]  
if verbose:
    print("Distance pairs")
    for l0 in l0_range():
        for d0 in d_range(l0):
             print(f"{l0} -> {l0+direction_to_lane_change(d0)}")

def s0_range():
    return range(N_SPEEDS)
def s1_range(index):
    return range(max(0,index-N_SPEED_DEVIATION), min(N_SPEEDS,index+N_SPEED_DEVIATION+1))
if verbose:
    print("Speed pairs")
    for s0 in s0_range():
        for s1 in s1_range(s0):
             print(f"{s0} -> {s1}")

#state*action space
action_range = [(l0,s0,d0,l0+direction_to_lane_change(d0),s1,d1) for l0 in l0_range() for s0 in s0_range() for d0 in d_range(l0) for s1 in s1_range(s0) for d1 in d_range(l0+direction_to_lane_change(d0)) ]
if verbose:
    print("state by action space", len(action_range))
    for (l0,s0,d0,l1,s1,d1) in action_range:
        print(f"{l0} {s0} {d0} -> {l1} {s1} {d1}")

def get_index(l0,s0,d0):
    return l0*N_SPEEDS*N_DIRECTIONS+s0*N_DIRECTIONS+d0


In [ ]:
#distance between lanes in subsequent time steps
def make_lane_distance_columns(l0,l1):
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1)
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1)
    return ((x_col1-x_col0).pow(2) + (y_col1-y_col0).pow(2)).sqrt().fill_null(strategy="forward").alias(f"d_l{l0}_l{l1}")
lane_distance_columns = [make_lane_distance_columns(l0,l1) for (l0,l1) in distance_pairs]

#distance between lanes in subsequent time steps
def make_velocity_change_column(l0,s0,d0,l1,s1,d1):
    l2 = l1+direction_to_lane_change(d1)
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    x_col0 = pl.col("x_lane"+str(l0))
    x_col1 = pl.col("x_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    x_col2 = pl.col("x_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    y_col0 = pl.col("y_lane"+str(l0))
    y_col1 = pl.col("y_lane"+str(l1)).shift(-1).fill_null(strategy="forward")
    y_col2 = pl.col("y_lane"+str(l2)).shift(-2).fill_null(strategy="forward")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    d12_col = pl.col(f"d_l{l1}_l{l2}").shift(-1).fill_null(strategy="forward")
    # If we want to account for acceleration and breaking separately, we should not bake the speed into this
    # return ((speed1*(x_col2-x_col1)/d12_col - speed0*(x_col1-x_col0)/d01_col).pow(2) + 
    #         (speed1*(y_col2-y_col1)/d12_col - speed0*(y_col1-y_col0)/d01_col).pow(2)).sqrt().fill_null(pl.lit(0)).alias(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    return ((speed0*(x_col2-x_col1)/d12_col - speed0*(x_col1-x_col0)/d01_col).pow(2) + 
            (speed0*(y_col2-y_col1)/d12_col - speed0*(y_col1-y_col0)/d01_col).pow(2)).sqrt().fill_null(pl.lit(0)).alias(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    
velocity_change_columns = [make_velocity_change_column(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]
#make name functions for these
velocity_change_columns_names = [f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}" for (l0,s0,d0,l1,s1,d1) in action_range]

#average speed
def make_average_speed(s0,s1):
    speed0 = index_to_speed(s0)
    speed1 = index_to_speed(s1)
    return (0.5*(pl.lit(speed0) + pl.lit(speed1))).alias(f"as_s{s0}_s{s1}")
average_speeds_columns = [make_average_speed(s0,s1) for s0 in s0_range() for s1 in s1_range(s0)]

#time cost
def make_time_costs(l0,l1,s0,s1):
    av_speed = pl.col(f"as_s{s0}_s{s1}")
    d01_col = pl.col(f"d_l{l0}_l{l1}")
    return pl.when(d01_col == 0).then(MAX_COST_VALUE).otherwise(d01_col/av_speed).alias(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
time_costs_columns = [make_time_costs(l0,l1,s0,s1) for l0 in l0_range() for l1 in l1_range(l0) for s0 in s0_range() for s1 in s1_range(s0)]

def make_acceleration(l0,s0,d0,l1,s1,d1):
    velocity_change = pl.col(f"fc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    time_cost = pl.col(f"tc_l{l0}_{l1}_s{s0}_s{s1}")
    return (velocity_change/time_cost).alias(f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
    #debug
    #return pl.lit( get_index(l0,s0,d0) + 10000*(get_index(l1,s1,d1)+111) ).alias(f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}")
acceleration_columns = [make_acceleration(l0,s0,d0,l1,s1,d1) for (l0,s0,d0,l1,s1,d1) in action_range]

opt_ldf = track_df.lazy().with_columns(lane_distance_columns).with_columns(velocity_change_columns).with_columns(average_speeds_columns).with_columns(time_costs_columns).with_columns(acceleration_columns)
opt_df  = opt_ldf .collect()
opt_df 

x_m,y_m,w_tr_right_m,w_tr_left_m,x_dir,y_dir,width,norm_dir,x_udir,y_udir,x_lane0,x_lane1,x_lane2,x_lane3,x_lane4,x_lane5,x_lane6,x_lane7,x_lane8,x_lane9,x_lane10,x_lane11,x_lane12,x_lane13,x_lane14,y_lane0,y_lane1,y_lane2,y_lane3,y_lane4,y_lane5,y_lane6,y_lane7,y_lane8,y_lane9,y_lane10,y_lane11,…,acc_l14s17d1_l13s19d1,acc_l14s17d1_l13s19d2,acc_l14s18d0_l14s16d0,acc_l14s18d0_l14s16d1,acc_l14s18d0_l14s17d0,acc_l14s18d0_l14s17d1,acc_l14s18d0_l14s18d0,acc_l14s18d0_l14s18d1,acc_l14s18d0_l14s19d0,acc_l14s18d0_l14s19d1,acc_l14s18d1_l13s16d0,acc_l14s18d1_l13s16d1,acc_l14s18d1_l13s16d2,acc_l14s18d1_l13s17d0,acc_l14s18d1_l13s17d1,acc_l14s18d1_l13s17d2,acc_l14s18d1_l13s18d0,acc_l14s18d1_l13s18d1,acc_l14s18d1_l13s18d2,acc_l14s18d1_l13s19d0,acc_l14s18d1_l13s19d1,acc_l14s18d1_l13s19d2,acc_l14s19d0_l14s17d0,acc_l14s19d0_l14s17d1,acc_l14s19d0_l14s18d0,acc_l14s19d0_l14s18d1,acc_l14s19d0_l14s19d0,acc_l14s19d0_l14s19d1,acc_l14s19d1_l13s17d0,acc_l14s19d1_l13s17d1,acc_l14s19d1_l13s17d2,acc_l14s19d1_l13s18d0,acc_l14s19d1_l13s18d1,acc_l14s19d1_l13s18d2,acc_l14s19d1_l13s19d0,acc_l14s19d1_l13s19d1,acc_l14s19d1_l13s19d2
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3.439354,-0.495322,6.556,6.536,5.862596,8.10221,13.092,10.000792,0.586213,0.810157,8.742641,7.985028,7.227416,6.469804,5.712191,4.954579,4.196966,3.439354,2.681742,1.924129,1.166517,0.408904,-0.348708,-1.10632,-1.863933,-4.332673,-3.78448,-3.236287,-2.688094,-2.139901,-1.591708,-1.043515,-0.495322,0.052871,0.601064,1.149257,1.69745,…,708.088882,2446.263336,680.337829,1265.20595,350.168093,1184.776683,0.308034,1197.898672,370.166286,1317.925335,1243.675878,668.772446,2192.81328,1164.609368,344.215428,2242.424472,1177.505407,0.313391,2345.158906,1295.491283,363.873663,2502.868469,720.334181,1396.215775,370.166286,1317.925335,0.343191,1334.617734,1372.455508,708.088882,2446.263336,1295.491283,363.873663,2502.868469,1311.896937,0.349159,2612.817545
6.370784,3.555763,6.558,6.537,5.862596,8.10221,13.095,10.000792,0.586213,0.810157,11.675286,10.9175,10.159714,9.401928,8.644142,7.886356,7.12857,6.370784,5.612998,4.855212,4.097426,3.33964,2.581854,1.824068,1.066282,-0.282468,0.265851,0.81417,1.362488,1.910807,2.459126,3.007444,3.555763,4.104082,4.6524,5.200719,5.749038,…,708.141558,2447.036362,680.395488,1265.544066,350.19776,1185.154673,0.289495,1198.307706,370.197645,1318.348668,1244.025484,668.822199,2193.500879,1165.005075,344.241003,2243.168687,1177.935399,0.255518,2345.951785,1295.934649,363.900694,2503.700721,720.395229,1396.597137,370.197645,1318.348668,0.322536,1335.073453,1372.850485,708.141558,2447.036362,1295.934649,363.900694,2503.700721,1312.376005,0.284681,2613.700916
9.30195,7.606888,6.56,6.538,5.862087,8.102292,13.098,10.00056,0.586176,0.810184,14.607844,13.849859,13.091874,12.333889,11.575905,10.81792,10.059935,9.30195,8.543965,7.78598,7.027995,6.270011,5.512026,4.754041,3.996056,3.768022,4.316432,4.864841,5.41325,5.96166,6.510069,7.058479,7.606888,8.155297,8.703707,9.252116,9.800526,…,708.14869,2447.575985,680.407831,1265.795483,350.204102,1185.450818,0.270473,1198.633699,370.204348,1318.680927,1244.23734,668.828935,2193.979945,1165.256437,344.244478,2243.694364,1178.212727,0.271182,2346.514237,1296.216731,363.904369,2504.288848,720.408297,1396.882736,370.204348,1318.680927,0.301343,1335.436652,1373.091388,708.14869,2447.575985,1296.216731,363.904369,2504.288848,1312.684986,0.302132,2614.327563
12.232871,11.658055,6.562,6.539,5.861618,8.10238,13.101,10.000356,0.586141,0.810209,17.540146,16.781964,16.023782,15.2656,14.507417,13.749235,12.991053,12.232871,11.474689,10.716507,9.958325,9.200142,8.44196,7.683778,6.925596,7.818539,8.367041,8.915544,9.464046,10.012548,10.56105,11.109553,11.658055,12.206557,12.75506,13.303562,13.852064,…,708

In [6]:
path = Path(FOLDER + f"\\opt_{N_LANES}.parquet")
#adf1.drop(velocity_change_columns_names).write_parquet(path)
opt_df.write_parquet(path)

In [7]:
#Tools to navigate the cost array
def make_cost_array(): return np.full((N_LANES*N_SPEEDS*N_DIRECTIONS,N_LANES*N_SPEEDS*N_DIRECTIONS,N_TIME), MAX_COST_VALUE)
#costs_arr = make_cost_array()

tc_arr = make_cost_array()
acc_arr = make_cost_array()
as_arr = make_cost_array()

def state_cost_tuple_to_index(l0,s0,d0):
    index = l0*N_SPEEDS*N_DIRECTIONS + s0*N_DIRECTIONS + d0 
    return index

for (l0,s0,d0,l1,s1,d1) in action_range:
    index0 = state_cost_tuple_to_index(l0,s0,d0) 
    index1 = state_cost_tuple_to_index(l1,s1,d1)
    tc_arr[index0][index1] = opt_df[f"tc_l{l0}_{l1}_s{s0}_s{s1}"].to_numpy()
    acc_arr[index0][index1] = opt_df[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"].to_numpy()
    as_arr[index0][index1] = opt_df[f"as_s{s0}_s{s1}"].to_numpy()

In [8]:
tc_arr = tc_arr.transpose(2,0,1)
as_arr = as_arr.transpose(2,0,1)
acc_arr = acc_arr.transpose(2,0,1)

In [ ]:
np.save(Path(FOLDER + f"\\tc_{N_LANES}"), tc_arr)
np.save(Path(FOLDER + f"\\as_{N_LANES}"), as_arr)
np.save(Path(FOLDER + f"\\acc_{N_LANES}"), acc_arr)

In [ ]:
#Test the frame has indeed the right entries
test = False
if test:
    at = acc_arr
    for l0 in range(N_LANES):
        for s0 in range(N_SPEEDS):
            for d0 in range(N_DIRECTIONS):
                index0 = get_index(l0,s0,d0)
                for l1 in range(N_LANES):
                    for s1 in range(N_SPEEDS):
                        for d1 in range(N_DIRECTIONS):
                            index1 = get_index(l1,s1,d1)
                            entry1 = at[0][index0][index1]
                            name = f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"
                            if name in opt_df.columns:
                                entry2 = opt_df[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"][0]
                                print(entry1==entry2)
                            else:
                                entry2 = MAX_COST_VALUE
                                print(entry1==entry2)


In [ ]:
(l0,s0,d0,l1,s1,d1) =(0, 0, 0, 0, 1, 0)
opt_df[f"acc_l{l0}s{s0}d{d0}_l{l1}s{s1}d{d1}"]

acc_l0s0d0_l0s1d0
f64
10.199067
10.198702
10.198964
10.199195
10.199399
…
10.196587
10.196987
10.197388
